In [36]:
!pip install torch torchvision transformers tdqm clean-text

     |████████████████████████████████| 175 kB 9.9 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 3.9 MB/s  eta 0:00:01
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171047 sha256=7dc07996a0a6c34a64775aa2ac92c68cbdbfc9257202409211828749576b83ac
  Stored in directory: /Users/lijiayi/Library/Caches/pip/wheels/fa/7a/e9/22dd0515e1bad255e51663ee513a2fa839c95934c5fc301090
Successfully built emoji


# Loading data from train-data, dev-data, test-data

In [1]:
import json
import os
from datetime import datetime

In [2]:
root_dir = '/Users/lijiayi/Desktop/project-data/'
train_ids_path = '/Users/lijiayi/Desktop/project-data/train.data.txt'
train_label_path = '/Users/lijiayi/Desktop/project-data/train.label.txt'
dev_ids_path = '/Users/lijiayi/Desktop/project-data/dev.data.txt'
dev_label_path = '/Users/lijiayi/Desktop/project-data/dev.label.txt'
test_ids_path = '/Users/lijiayi/Desktop/project-data/test.data.txt'

train_ids = open(train_ids_path, 'r').readlines()
train_labels = open(train_label_path, 'r').readlines()
dev_ids = open(dev_ids_path, 'r').readlines()
dev_labels = open(dev_label_path, 'r').readlines()
test_ids = open(test_ids_path, 'r').readlines()

In [3]:
def train_data_preparation(train_ids, train_labels):
    tweets = []
    texts = []
    int_labels = []
    for ids, label in zip(train_ids, train_labels):
        ## finding the tweet object and text with corresponding id
        id_list = ids.strip().split(",")
        tweet_objects = []
        tweet_texts = []
        ## check if the source tweet has been crawled ans stored in os, otherwise skip
        if os.path.exists(root_dir + 'train-data/' + id_list[0] + '.json'):
            for tid in id_list:
                data_file_path = root_dir + 'train-data/' + tid + '.json'
                ## check if each tweet has crawled data, otherwise skip 
                if os.path.exists(data_file_path):
                    tweet = open(data_file_path)
                    data = json.load(tweet)
                    tweet_objects.append(data)
                    tweet.close
            ## sort the tweet objects by timestamp
            tweet_objects = sorted(tweet_objects, key=lambda x: datetime.timestamp(datetime.strptime(x['created_at'], '%Y-%m-%dT%H:%M:%S.%fZ')))
            tweets.append(tweet_objects)

            ## put texts together as a context
            for obj in tweet_objects:
                tweet_texts.append(obj['text'])
            texts.append(tweet_texts)

            ## convert string label to int label
            int_label = 0 if label.strip() == 'nonrumour' else 1
            int_labels.append(int_label)

    return tweets, texts, int_labels

In [4]:
def dev_data_preparation(dev_ids, dev_labels):
    tweets = []
    texts = []
    int_labels = []
    for ids, label in zip(dev_ids, dev_labels):
        ## finding the tweet object and text with corresponding id
        id_list = ids.strip().split(",")
        tweet_objects = []
        tweet_texts = []
        ## check if the source tweet has been crawled ans stored in os, otherwise skip
        if os.path.exists(root_dir + 'dev-data/' + id_list[0] + '.json'):
            for tid in id_list:
                data_file_path = root_dir + 'dev-data/' + tid + '.json'
                ## check if each tweet has crawled data, otherwise skip 
                if os.path.exists(data_file_path):
                    tweet = open(data_file_path)
                    data = json.load(tweet)
                    tweet_objects.append(data)
                    tweet.close
            ## sort the tweet objects by timestamp
            tweet_objects = sorted(tweet_objects, key=lambda x: datetime.timestamp(datetime.strptime(x['created_at'], '%Y-%m-%dT%H:%M:%S.%fZ')))
            tweets.append(tweet_objects)

            ## put texts together as a context
            for obj in tweet_objects:
                tweet_texts.append(obj['text'])
            texts.append(tweet_texts)

            ## convert string label to int label
            int_label = 0 if label.strip() == 'nonrumour' else 1
            int_labels.append(int_label)

    return tweets, texts, int_labels

In [5]:
def test_data_preparation(test_ids):
    tweets = []
    texts = []
    for ids in test_ids:
        ## finding the tweet object and text with corresponding id
        id_list = ids.strip().split(",")
        tweet_objects = []
        tweet_texts = []
        ## check if the source tweet has been crawled ans stored in os, otherwise skip
        if os.path.exists(root_dir + 'tweet-objects/' + id_list[0] + '.json'):
            for tid in id_list:
                data_file_path = root_dir + 'tweet-objects/' + tid + '.json'
                ## check if each tweet has crawled data, otherwise skip 
                if os.path.exists(data_file_path):
                    tweet = open(data_file_path)
                    data = json.load(tweet)
                    tweet_objects.append(data)
                    tweet.close
            ## sort the tweet objects by timestamp
            tweet_objects = sorted(tweet_objects, key=lambda x: datetime.timestamp(datetime.strptime(x['created_at'], '%a %b %d %H:%M:%S +0000 %Y')))
            tweets.append(tweet_objects)

            ## put texts together as a context
            for obj in tweet_objects:
                tweet_texts.append(obj['text'])
            texts.append(tweet_texts)

    return tweets, texts

In [6]:
train_tweets, train_texts, train_int_labels = train_data_preparation(train_ids, train_labels)
dev_tweets, dev_texts, dev_int_labels = dev_data_preparation(dev_ids, dev_labels)
test_tweets, test_texts = test_data_preparation(test_ids)

In [7]:
print(len(train_texts), len(train_int_labels))
print(len(dev_texts), len(dev_int_labels))
print(len(test_texts))

1560 1560
535 535
558


In [9]:
train_texts[0]

['4. Can eating garlic help prevent infection with the new coronavirus? #COVID19Malaysia https://t.co/q133xXBiwl',
 '5. Can regularly rinsing your nose with saline help prevent infection with the new coronavirus? https://t.co/ccMjhhD7BK',
 '6. Do vaccines against pneumonia protect you against the new coronavirus? https://t.co/wL0mlEqU95',
 '7. Can spraying alcohol or chlorine all over your body kill the new coronavirus? #Chamber https://t.co/zunVR7Ht0V',
 '8. How effective are thermal scanners in detecting people infected with the new coronavirus? https://t.co/nyLOyKAb1H',
 '9. Can an ultraviolet disinfection lamp kill the new coronavirus? https://t.co/ZrlllbkIjm',
 '10. Are hand dryers effective in killing the new coronavirus? https://t.co/cSDKXO1bGr',
 '11. The new coronavirus CANNOT be transmitted through mosquito bites. https://t.co/ZRL8bjRkpl',
 '12. Taking a hot bath does not prevent the new coronavirus disease https://t.co/bICOqSTOuD',
 '13. Cold weather and snow CANNOT kill the

# Text preprocress for train-data, dev-data, test-data

In [10]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

In [14]:
from cleantext import clean

def text_preprocess(tweet_text):
    new_text = []
    for t in tweet_text.split(" "):
        if t in set(stopwords.words('english')):
            continue
        t = clean(t, no_emoji=True)
        t = t.lstrip('@') if t.startswith('@') and len(t) > 1 else t
        t = t.lstrip('#') if t.startswith('#') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    #new_text.append('[SEP] ')
    return " ".join(new_text)

In [15]:
def create_dataframe(_texts, _labels):
    total_list = []
    for event, label in zip(_texts, _labels):
        temp_dict = {}
        source = event[0]
        reply_list = event[1:]
        
        text_concat = []
        for reply in reply_list:   
            text_concat.append(text_preprocess(reply))
        reply_concat = " ".join(text_concat)
        
        temp_dict['source']= text_preprocess(source)
        temp_dict['reply'] = reply_concat
        
        #temp_dict['text'] = text_concat
        temp_dict['label'] = label
        event_ids = _texts.index(event)
        temp_dict['event_id']  = event_ids
        total_list.append(temp_dict)
        
    total_df = pd.DataFrame(total_list)
    return total_df

In [16]:
def create_test_dataframe(_texts):
    total_list = []
    for event in _texts:
        temp_dict = {}
        source = event[0]
        reply_list = event[1:]
        
        text_concat = []
        for reply in reply_list:   
            text_concat.append(text_preprocess(reply))
        reply_concat = " ".join(text_concat)
        
        temp_dict['source']= text_preprocess(source)
        temp_dict['reply'] = reply_concat
            
        #temp_dict['text'] = text_concat
        event_ids = _texts.index(event)
        temp_dict['event_id']  = event_ids
        total_list.append(temp_dict)
        
    total_df = pd.DataFrame(total_list)
    return total_df

In [17]:
train_df = create_dataframe(train_texts, train_int_labels)
dev_df = create_dataframe(dev_texts, dev_int_labels)
test_df = create_test_dataframe(test_texts)

In [18]:
train_df

,source,reply,label,event_id
0,4. can eating garlic help prevent infection ne...,5. can regularly rinsing nose saline help prev...,0,0
1,french police chief killed charliehebdo attack...,telegraph how sad. telegraph telegraphnews the...,1,1
2,coronavirus disease (covid-19) advice public: ...,infection control suspected confirmed covid-19...,0,2
3,ottawa police confirm multiple suspects shooti...,"wsj killers go berserk cornered. henceforth, ...",0,3
4,primary focus government alleviate suffering/d...,,0,4
...,...,...,...,...
1555,4. it cannot transmitted goods manufactured ch...,corona\nall graphics taken world health organi...,0,1555
1556,desperate ted cruz claims planned parenthood s...,bipartisanism desperate! bipartisanism cruz o...,1,1556
1557,"""thoughts prayers enough."" pres. obama speaks ...",.@abc anyone else noticed mass shootings skyro...,1,1557
1558,police surrounded building suspected charliehe...,nbcnews bury hole nbcnews wikileakstruck cops ...,0,1558


In [19]:
dev_df

,source,reply,label,event_id
0,covid-19 fact:\nare hand dryers effective kill...,"weatherbug they are, fact, germ-breeding facto...",0,0
1,"atruchecks expect result husband's "" pending""a...",ewart_lynne atruchecks hi luck? in boat? ot_ch...,0,1
2,i've read lot corona virus lately i think i sh...,what coronavirus? coronavirus large family vir...,0,2
3,"every news outlet using headlines like,\n""are ...","tuckyaalto apparently, headline question, answ...",0,3
4,researcher naskrecki encounter goliath birdeat...,harvard naskrecki eu tenho uma dessas em casa ...,0,4
...,...,...,...,...
530,what are the treatment options for covid-19 (i...,antibiotics work. antibiotics used means preve...,0,454
531,"after speculation he's arrested, banksy debuts...",artnet xklamation story saying someone vandali...,1,531
532,*your questions answered*\n*reply number get c...,s?\n14. can i catch covid-19 infected surfaces...,0,532
533,">#anonymous operation kkk >ku klux klan, we ne...",anonymousvideo http grannyrosie3 anonymousvide...,1,533


In [20]:
test_df

,source,reply,event_id
0,how does covid-19 spread? http http,"wcco thanks, wcco! you station i trust media p...",0
1,"brain_warrior if don't believe me, look change...","brain_warrior tell people ""protesting"". brain_...",1
2,q. how covid-19 influenza viruses similar?\n#c...,q. how covid-19 influenza viruses different?\n...,2
3,una de les q&a coronaviruses de la pagina web ...,aquesta informacio es basa sobre tot en un art...,3
4,malaikajabali benjaminpdixon we can't forget r...,"malaikajabali benjaminpdixon not mention, neve...",4
...,...,...,...
553,ex-marlboro man dies smoking-related disease http,"""@washingtonpost: ex-marlboro man dies smoking...",553
554,holy shit. doritos flavored mountain dew.\nall...,boogie2988 boogie2988 dewritos boogie2988 what...,554
555,banksy account joins cartoonists support charl...,"thepoke sadly work though thepoke ""@theazzo: n...",555
556,drtedros whowpro whosearo who_europe pahowho w...,drtedros whowpro whosearo who_europe pahowho w...,556


In [21]:
train_df.to_csv('train_data.csv',index=False)
dev_df.to_csv('dev_data.csv',index=False)
test_df.to_csv('test_data.csv',index=False)

# Dataset Preparation

In [22]:
from transformers import RobertaTokenizer, RobertaModel, BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
import torch
from nltk import TweetTokenizer

In [23]:
train_df.loc[0, 'reply']

'5. can regularly rinsing nose saline help prevent infection new coronavirus? http 6. do vaccines pneumonia protect new coronavirus? http 7. can spraying alcohol chlorine body kill new coronavirus? chamber http 8. how effective thermal scanners detecting people infected new coronavirus? http 9. can ultraviolet disinfection lamp kill new coronavirus? http 10. are hand dryers effective killing new coronavirus? http 11. the new coronavirus cannot transmitted mosquito bites. http 12. taking hot bath prevent new coronavirus disease http 13. cold weather snow cannot kill new coronavirus. http 14. covid-19 virus transmitted areas hot humid climates http 15. drinking alcohol protect covid-19 dangerous http 16. being able hold breath 10 seconds without coughing feeling discomfort does not mean free coronavirus disease (covid-19) lung disease. http 17. you recover coronavirus disease (covid-19). catching new coronavirus does not mean life. http 18. exposing sun temperatures higher 25c degrees do

In [24]:
class TweetDataset(Dataset):
    def __init__(self, dataframe, maxlen):
        self.dataset = dataframe
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.maxlen = maxlen
        
    def __len__(self):
        return len(self.dataset)
        
    def __getitem__(self, index):
        source = self.dataset.loc[index, 'source']
        reply = self.dataset.loc[index,'reply']
        #text = self.dataset.loc[index, 'text']
        label = self.dataset.loc[index, 'label']
        #source = text_preprocess(source)
        #reply = text_preprocess(reply)
        
        inputs = self.tokenizer.encode_plus(
          source,
          reply,
          #text,
          padding='max_length',
          add_special_tokens=True,
          return_attention_mask=True,
          max_length=self.maxlen,
          return_token_type_ids=True,
          truncation=True,
          )
        token_ids = inputs['input_ids']
        seg_ids = inputs['token_type_ids']
        mask = inputs['attention_mask']

        tokens_ids_tensor = torch.tensor(token_ids,dtype=torch.long)
        attn_mask = torch.tensor(mask, dtype=torch.long)
        seg_ids_tensor = torch.tensor(seg_ids, dtype=torch.long)
        
        return tokens_ids_tensor, attn_mask, seg_ids_tensor, label

In [25]:
class TestDataset(Dataset):
    def __init__(self, dataframe, maxlen):
        self.dataset = dataframe
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.maxlen = maxlen
        
    def __len__(self):
        return len(self.dataset)
        
    def __getitem__(self, index):
        source = self.dataset.loc[index, 'source']
        reply = self.dataset.loc[index,'reply']
        #source = text_preprocess(source)
        #reply = text_preprocess(reply)

        inputs = self.tokenizer.encode_plus(
          source,
          reply,
          padding='max_length',
          add_special_tokens=True,
          return_attention_mask=True,
          max_length=self.maxlen,
          return_token_type_ids=True,
          truncation=True
          )
        token_ids = inputs['input_ids']
        seg_ids = inputs['token_type_ids']
        mask = inputs['attention_mask']

        tokens_ids_tensor = torch.tensor(token_ids,dtype=torch.long)
        attn_mask = torch.tensor(mask, dtype=torch.long)
        seg_ids_tensor = torch.tensor(seg_ids, dtype=torch.long)
        
        return tokens_ids_tensor, attn_mask, seg_ids_tensor

In [26]:
train_set = TweetDataset(train_df, maxlen=120)
dev_set = TweetDataset(dev_df, maxlen=120)
test_set = TestDataset(test_df, maxlen=120)

In [27]:
train_loader = DataLoader(train_set, batch_size = 64, num_workers = 2)
dev_loader = DataLoader(dev_set, batch_size = 64, num_workers = 2)
test_loader = DataLoader(test_set, batch_size = 64, num_workers = 2)

# Building Model

In [24]:
torch.cuda.empty_cache()
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel

#tokenizer = AutoTokenizer.from_pretrained("tristantristantristan/rumor")

class RumourModel(nn.Module):
    
    def __init__(self,model_name):
        super(RumorModel, self).__init__()
        #self.MODEL = 'cardiffnlp/twitter-roberta-base-sentiment'
        #self.bert_model = AutoModelForSequenceClassification.from_pretrained(self.MODEL)
        self.MODEL = 'bert-base-uncased'
        self.bert_model = BertModel.from_pretrained(self.MODEL)
        #self.bert_model = AutoModel.from_pretrained('cardiffnlp/twitter-roberta-base-2021-124m')
        #self.dropout = nn.Dropout(p=0.3)
        self.cls_layer = nn.Linear(768,1)

    def forward(self, ids, mask, seg_ids):
        outputs = self.bert_model(ids,attention_mask=mask,token_type_ids=seg_ids,return_dict=True)
        #pooled_output = outputs.pooler_output

        #pooled_output = torch.cat(tuple([hidden_states[:,i] for i in [-4, -3, -2, -1]]), dim=-1)
        #pooled_output = self.dropout(pooled_output)
        # classifier of course has to be 4 * hidden_dim, because we concat 4 layers

        #logits = self.cls_layer(pooled_output)

        last = outputs.last_hidden_state
        cls_rep = last[:,0]

        #cls_rep = self.dropout(pooled_output)
        logits = self.cls_layer(cls_rep)
        return logits

In [ ]:
# gpu = 0 #gpu ID

print("Creating the rumour model, initialised with pretrained BERT-BASE parameters...")
net = RumourModel()
# net.cuda(gpu) #Enable gpu support for the model
print("Done creating the runmour model.")

In [25]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.NLLLoss()
opti = optim.SGD(net.parameters(), lr = 0.005)

NameError: name 'net' is not defined

# Training Model

In [ ]:
import time
from tqdm import tqdm

def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):

    best_acc = 0
    st = time.time()
    for ep in tqdm(range(max_eps)):
        
        net.train()
        for it, (seq, attn_masks, seg_ids, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            seq, attn_masks, seg_ids, labels = seq.cuda(gpu), attn_masks.cuda(gpu), seg_ids.cuda(gpu), labels.cuda(gpu)

            #Obtaining the logits from the model
            logits = net(seq, attn_masks, seg_ids)

            #Computing loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()
              
            if it % 100 == 0:
                
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()

        
        dev_acc, dev_loss = evaluate(net, criterion, dev_loader, gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc, dev_loss))
        if dev_acc > best_acc:
            print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, dev_acc))
            best_acc = dev_acc
            torch.save(net.state_dict(), 'sstcls_{}.dat'.format(ep))

In [ ]:
def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    return acc

def evaluate(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    with torch.no_grad():
        for seq, attn_masks, labels in dataloader:
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
            logits = net(seq, attn_masks)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            count += 1

    return mean_acc / count, mean_loss / count

In [ ]:
num_epoch = 8

#fine-tune the model
train(net, criterion, opti, train_loader, dev_loader, num_epoch, gpu)

# Prediction

In [ ]:
gpu = 0
path = '/content/sstcls_0.dat'
mode_predict = RumourModel()
mode_predict.load_state_dict(torch.load(path))
mode_predict.cuda(gpu)

In [ ]:
def predict(model, dataloader, gpu):
    model.eval()

    #mean_acc, mean_loss = 0, 0
    count = 0
    prob_list = []
    predicted_label_list = []
    with torch.no_grad():
        for seq, attn_masks, seg_ids in dataloader:
            seq, attn_masks, seg_ids = seq.cuda(gpu), attn_masks.cuda(gpu), seg_ids.cuda(gpu)
            logits = model(seq, attn_masks, seg_ids)
            probs = torch.sigmoid(logits.unsqueeze(-1))
            soft_probs = (probs > 0.5).long()

            predicted_label_list.append(soft_probs.squeeze().cpu().detach().numpy())

#            acc = (soft_probs.squeeze() == labels)
 #           prob_list.append(soft_probs)
            #predicted_label_list.append(soft_probs.squeeze())
            #mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            #mean_acc += get_accuracy_from_logits(logits, labels)
            #count += 1

    return prob_list, predicted_label_list

In [ ]:
gpu = 0
prob_list, predicted_label_list = predict(model_predict, test_loader, gpu )

In [ ]:
import numpy as np
def output_label(test_df, label_list):
    output = label_list[0]
    for tensor in label_list[1:]:
        #temp_arr = tensor.cpu().detach().numpy()
        #print(temp_arr)
        output = np.concatenate((output, tensor))
        test_df['label'] = output
        #output_df = test_df.loc[['problem_num','label']]
    return test_df

In [ ]:
output_df = output_label(test_df, predicted_label_list)

In [ ]:
output_df = output_df[['event_id','label']]
output_df.columns = ['Id','Predicted']
output_df
output_df.to_csv('out.csv',index=False)